# Implementation of the redactle game

In [1]:
import numpy as np
from utils import propose_best_word

c:\Users\thoma\Anaconda\envs\redactle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialization

In [20]:
titre = "Le Rêve dans le pavillon rouge"
article = "Le Rêve dans le pavillon rouge écrit en l'espace de dix ans par Cao Xueqin est le dernier en date des quatre grands romans de la littérature classique chinoise, considéré par Mao Zedong comme l'une des fiertés de la Chine2. Il fut écrit au milieu du xviiie siècle durant la dynastie Qing. Il est considéré comme l'un des chefs-d’œuvre de la littérature chinoise, si ce n'est le plus prestigieux, et est généralement considéré comme l'apogée de l'art romanesque chinois (autobiographique, pour une part) ; à ce titre, l'immense œuvre fait partie de la collection UNESCO d'œuvres représentatives. L'exégèse du Rêve dans le pavillon rouge et tout ce qui s'y rapporte représente un pan entier de l'étude littéraire que l'on appelle « rougeologie »"

In [47]:
included_words = ["L'",
 'de',
  '(',
 ')',
 'en',
 ',',
 'est',
 'une',
 'le',
 'dans',
 'la',
 'à',
 'pour',
 'entre',
 'des',
 "d'",
 'qui',
 'Ces',
 'et',
 'sur',
 'les',
 "n'",
 ':',
 '«',
 'plus',
 '»',
 'du',
 'par',
 "s'",
 'y',
 'ainsi']

In [45]:
# for _ in range(20):
#     new_word = input()
#     if new_word not in included_words:
#         included_words.append(new_word)

## Update

In [26]:
titre = "Le Rêve dans le pavillon rouge"
article = "Le Rêve dans le pavillon rouge écrit en l'espace de dix ans par Cao Xueqin, est le dernier en date des quatre grands romans de la littérature classique chinoise, considéré par Mao Zedong comme l'une des fiertés de la Chine."

In [27]:
titre_masked = "Le <mask> dans le <mask> <mask>"
article_masked = "Le <mask> dans le <mask> <mask> <mask> en l'<mask> de <mask> <mask> par <mask> <mask>, est le <mask> en <mask> des <mask> <mask> <mask> de la <mask> <mask> <mask>, <mask> par <mask> <mask> comme l'une des <mask> de la <mask>."

In [31]:
tried_words = [',', 'Le', 'dans', 'le', "l'", "par", "est", "de", "comme", "une", "un", "<unk>", ".", "s"]

In [32]:
suggestion, tried_words = propose_best_word(article_masked, 5, tried_words)

In [33]:
suggestion

'l'

In [34]:
suggestion = 'considéré'

On cherche d'abord dans le titre

In [35]:
for i, (word, blank) in enumerate(zip(titre.split(), titre_masked.split())):
    if word == suggestion:
        titre_masked_list = titre_masked.split()
        titre_masked_list[i] = suggestion
        titre_masked = " ".join(titre_masked_list)

In [36]:
titre_masked

'Le <mask> dans le <mask> <mask>'

Puis dans l'article

In [37]:
print(article_masked)

for i, (word, blank) in enumerate(zip(article.split(), article_masked.split())):
    if word == suggestion:
        article_masked_list = article_masked.split()
        article_masked_list[i] = suggestion
        article_masked = " ".join(article_masked_list)

Le <mask> dans le <mask> <mask> <mask> en l'<mask> de <mask> <mask> par <mask> <mask>, est le <mask> en <mask> des <mask> <mask> <mask> de la <mask> <mask> <mask>, <mask> par <mask> <mask> comme l'une des <mask> de la <mask>.


In [38]:
print(article_masked)

Le <mask> dans le <mask> <mask> <mask> en l'<mask> de <mask> <mask> par <mask> <mask>, est le <mask> en <mask> des <mask> <mask> <mask> de la <mask> <mask> <mask>, considéré par <mask> <mask> comme l'une des <mask> de la <mask>.


In [39]:
tried_words

[',',
 'Le',
 'dans',
 'le',
 "l'",
 'par',
 'est',
 'de',
 'comme',
 'une',
 'un',
 '<unk>',
 '.',
 's',
 'l']